In [1]:
import sys
# !{sys.executable} -m pip install mendeleev
#######################################################
#Import packages
import numpy as np
import os
import re
from math import sin, cos, pi
from glob import glob
import subprocess
import pickle
from subprocess import call, check_output
import pandas as pd
# import psi4
from joblib import Parallel,effective_n_jobs,delayed
from time import time
import matplotlib.pyplot as plt
from plumbum.cmd import grep, awk

import shutil
import random
import sklearn
from shutil import copy
import csv
import h5py as h5
import seaborn as sns; sns.set(style="ticks", color_codes=True)

from sklearn.model_selection import train_test_split
import mendeleev
from tqdm import tqdm

In [2]:
radius_range=np.arange(0.6,2.9,0.1)

In [3]:
def writexyz(atom_A,atom_B,radius,filename):
    """
    Write a xyz coordinate to file

    parameters
    ----------
    atom_A: str
        First atom
        
    atom_B: str
        Second atom
    
    radius: float
        Radius of A and B
        
    filename: str
        Name of xyz file
    
    """
    coords=f"""2
    
{atom_A} {0:>8.4f} {0:>8.4f} {0:>8.4f}
{atom_B} {0:>8.4f} {0:>8.4f} {radius:>8.4f}
"""
    with open(filename,'w') as f:
        f.write(coords)

In [4]:
# Delete excessive extra files
def del_useless():
    '''
    Delete the extra files
    '''
    for root, dirs, files in os.walk(os.getcwd()):
        for file in files:
            for j in ['status','GssOrb','LprOrb','LoProp','guessorb','xmldump','RasOrb','SpdOrb']:
                if j in file:
    #                 print(root,dirs,file)
                    os.remove(os.path.join(root,file))

In [5]:
def gen_gateway(name,basis_set):
    string=f'''&GATEWAY 
coord={f'{name}.xyz'}
Basis = {basis_set}
Group = nosymm
End of Input
'''
    return string

def gen_seward():
    string=f'''&SEWARD
End of Input
'''
    return string

def gen_motra():
    string=f'''&MOTRA
'''
    return string

def gen_MP2():
    string="""&MBPT2 &END\n"""
    return string


def gen_scf():
    string=f"""&SCF &END
UHF
"""
    return string    


def gen_rasscf(e,o,i,spin,previous=None):
    start_string="""&RASSCF &END
Title= RASSCF
"""
    if previous!=None:
        fileorb=f"""FileOrb
{previous}
"""
    else:
        fileorb=''

    end_string=f"""
NACTEL
{e} 0 0
Inactive
{i}
RAS2
{o}
Symmetry
1
Spin
{spin}
orblisting
all
ITERation
200 100
CIMX
200
SDAV
500
"""
    return start_string+fileorb+end_string 

def gen_caspt2():
    string="""&CASPT2 &END
Imaginary Shift
0.0

"""
    return string    





In [6]:
with open("diatomics.txt") as f:
    diatomics=[i.strip('\n').split('_') for i in f.readlines()]

In [7]:
diatomics

[['B', 'B'],
 ['B', 'Be'],
 ['B', 'C'],
 ['B', 'F'],
 ['B', 'H'],
 ['B', 'Li'],
 ['B', 'N'],
 ['B', 'O'],
 ['Be', 'Be'],
 ['Be', 'C'],
 ['Be', 'F'],
 ['Be', 'H'],
 ['Be', 'Li'],
 ['Be', 'N'],
 ['Be', 'O'],
 ['C', 'C'],
 ['C', 'F'],
 ['C', 'H'],
 ['C', 'Li'],
 ['C', 'N'],
 ['C', 'O'],
 ['F', 'F'],
 ['F', 'H'],
 ['F', 'Li'],
 ['F', 'N'],
 ['F', 'O'],
 ['H', 'H'],
 ['H', 'Li'],
 ['H', 'N'],
 ['H', 'O'],
 ['Li', 'Li'],
 ['Li', 'N'],
 ['Li', 'O'],
 ['N', 'N'],
 ['N', 'O'],
 ['O', 'O']]

In [8]:
def write_input(dirname,basis_set,radius,idxr,e,o,i,spin):
    """
    Create OpenMolcas Input

    parameters:
    -----------
    dirname: str
        Name of directory
    
    basis_set: str
        Basis set of choice
    
    radius: float
        Radius 
        
    idxr: int
        Index of radius

    e: int
        Active electrons
        
    o: int
        Active orbitals
        
    i: int
        Inactive orbitals

    spin: int
        Spin multiplicity
    """
    with open(os.path.join(dirname,f'{radius:.2f}',f'{radius:.2f}.input'),'wb') as g:
        g.write(gen_gateway(f"{radius:.2f}",basis_set).encode())
        g.write(gen_seward().encode())
        g.write(gen_scf().encode())   
        g.write(gen_MP2().encode())   
        # Choose active space and inactive orbitals
        #g.write(gen_rasscf(name,2,2,int((i/2)-1)).encode())
        if idxr==0:
            g.write(gen_rasscf(e,o,i,spin,previous=None).encode()) 
        else:
    
            previous=os.path.join(dirname,f'{radius_range[idxr-1]:.2f}',f"{radius_range[idxr-1]:.2f}.RasOrb")
            g.write(gen_rasscf(e,o,i,spin,previous=previous).encode()) # int((i/2)-1)
        g.write(gen_caspt2().encode())    

In [9]:
basis_set="ANO-RCC-MB"
top=os.path.join('/home/grierjones/DDCASPT2/delta_ML')
for A,B in tqdm(diatomics):
    A_ele=getattr(mendeleev,A).electrons
    B_ele=getattr(mendeleev,B).electrons

    dirname=os.path.join(top,''.join((A,B)))
    if os.path.exists(dirname)==False:
        os.mkdir(dirname)
        
    for idr, r in tqdm(enumerate(radius_range)):
        raddir=os.path.join(dirname,f"{r:.2f}")
        print(f"raddir {raddir}")
        if os.path.exists(raddir)==False:
            os.mkdir(raddir)
        writexyz(A,B,r,os.path.join(dirname,f"{r:.2f}/{r:.2f}.xyz"))
        
        if (A_ele+B_ele)%2==0:
            allorbs=(A_ele+B_ele)//2
            active_o=2
            active_e=2
            inactive=allorbs-active_o
            spin=1
            if allorbs>1:
                write_input(dirname,basis_set,r,idr,active_o,active_o,inactive,spin)
            else:
                write_input(dirname,basis_set,r,idr,active_o,active_o,0,spin)
        else:
            allorbs=(A_ele+B_ele+1)//2
            active_o=2
            active_e=3
            inactive=allorbs-active_o
            spin=2
            write_input(dirname,basis_set,r,idr,active_e,active_o,inactive,spin)

        
        if os.getcwd()!=raddir:    
            os.chdir(raddir)
        print(os.getcwd())
        # Run
        call(['pymolcas','-new','-clean',f'{r:.2f}.input', '-oe', f'{r:.2f}.output'])

        # Back to top dir

        if os.getcwd()!=os.path.join(dirname):
            os.chdir(os.path.join(dirname))
        print(os.getcwd())
    print(os.getcwd())

  0%|                                                                                                                                                                                       | 0/36 [00:00<?, ?it/s]
0it [00:00, ?it/s]<string>:392: SyntaxWarning: invalid escape sequence '\.'
<string>:401: SyntaxWarning: invalid escape sequence '\.'
<string>:412: SyntaxWarning: invalid escape sequence '\d'
<string>:707: SyntaxWarning: invalid escape sequence '\s'


raddir /home/grierjones/DDCASPT2/delta_ML/BB/0.60
/home/grierjones/DDCASPT2/delta_ML/BB/0.60



1it [00:01,  1.23s/it]

/home/grierjones/DDCASPT2/delta_ML/BB
raddir /home/grierjones/DDCASPT2/delta_ML/BB/0.70
/home/grierjones/DDCASPT2/delta_ML/BB/0.70


<string>:392: SyntaxWarning: invalid escape sequence '\.'
<string>:401: SyntaxWarning: invalid escape sequence '\.'
<string>:412: SyntaxWarning: invalid escape sequence '\d'
<string>:707: SyntaxWarning: invalid escape sequence '\s'

2it [00:02,  1.24s/it]

...

/home/grierjones/DDCASPT2/delta_ML/OO
raddir /home/grierjones/DDCASPT2/delta_ML/OO/2.70
/home/grierjones/DDCASPT2/delta_ML/OO/2.70


<string>:392: SyntaxWarning: invalid escape sequence '\.'
<string>:401: SyntaxWarning: invalid escape sequence '\.'
<string>:412: SyntaxWarning: invalid escape sequence '\d'
<string>:707: SyntaxWarning: invalid escape sequence '\s'

22it [00:27,  1.22s/it]

/home/grierjones/DDCASPT2/delta_ML/OO
raddir /home/grierjones/DDCASPT2/delta_ML/OO/2.80
/home/grierjones/DDCASPT2/delta_ML/OO/2.80


<string>:392: SyntaxWarning: invalid escape sequence '\.'
<string>:401: SyntaxWarning: invalid escape sequence '\.'
<string>:412: SyntaxWarning: invalid escape sequence '\d'
<string>:707: SyntaxWarning: invalid escape sequence '\s'

23it [00:28,  1.26s/it]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [18:10<00:00, 30.28s/it]

/home/grierjones/DDCASPT2/delta_ML/OO
/home/grierjones/DDCASPT2/delta_ML/OO
